### Import Libraries

In [1]:
import cryptocompare
import requests
import json
from pprint import pprint
import datetime
from queue import Queue
import threading
import time

### Declare Stats of Interest

In [2]:
# Use Cryptocompare live social status function to get the data from various social websites
# Social Status Keys- 'General', 'CryptoCompare', 'Twitter', 'Reddit', 'Facebook', 'CodeRepository'

############## Keys of Interest #################
# Reddit- Points, comments_per_day, name, posts_per_day, posts_per_hour, subscribers, name
# Twitter- Points, followers, statuses
# General- CoinName
# Facebook- Points, likes, talking_about
# CryptoCompare- Points, Followers, PageViews, PageViewsSplit
# CodeRepository- TODO LATER, not of interest yet

# Stats of interest for each social site- change to suit your needs
redditstats = ['Points', 'comments_per_day', 'name', 'posts_per_day', 'posts_per_hour', 'subscribers']
twitterstats = ['Points', 'followers', 'statuses']
generalstats = ['CoinName']
facebookstats = ['Points', 'likes', 'talking_about']
cryptocomparestats = ['Points', 'Followers', 'PageViews', 'PageViewsSplit']

# Create a master dictionary for all important stats
masterstats = {}
masterstats['Reddit'] = redditstats
masterstats['Twitter'] = twitterstats
masterstats['General'] = generalstats
masterstats['Facebook'] = facebookstats
masterstats['CryptoCompare'] = cryptocomparestats

### Define Function to Grab Live Social Status From CryptoCompare API

In [3]:
# Grab cryptocompare's id and use it to grab live social status data
def live_social_status(symbol, symbol_id_dict={}):
    if not symbol_id_dict:
        symbol_id_dict = {
            'BTC': 1182,
            'ETH': 7605,
            'LTC': 3808
        }
    symbol_id = symbol_id_dict[symbol.upper()]
    url = 'https://www.cryptocompare.com/api/data/socialstats/?id={}'\
            .format(symbol_id)
    page = requests.get(url)
    data = page.json()['Data']
    return data

### Grab List of All Coins From Cryptocompare and Extract the Symbols
---
**Note:** symbols only contains 10 elements as of now (spliced from 0 to 10) for prettiness sake. Remove '[0:10]' if you want to include all the coins.

In [4]:
# Raw data from cryptocompare
coinlist = cryptocompare.get_coin_list()
print('Finished Grabbing List of All Coins')

# Dictionary of cryptocompare Ids. Format- {Coin: ID} 
symbol_id_dict = {}

for coin in coinlist.keys():
    symbol_id_dict[coin] = coinlist[coin]['Id']

symbols = list(symbol_id_dict.keys())[0:10]
print('Finished Extracing Symbols')

Finished Grabbing List of All Coins
Finished Extracing Symbols


### Define Functions to Create the Master Dictionary
---
The Master dictionary stores the data in a nested dictinoary format.

`{Date:   
    {Coins:  
        {Reddit: [stats],  
        Facebook: [stats], etc...}  
    }  
}  `

get_stats() grabs the necessary data for each coin and stores them in a dictionary

To speed up the process, we implement multithreading. Essentially, it allows multiple jobs to be worked on at the same time, instead of one job after another.

worker() does most of the work. We can now have multiple threads doing various tasks (namely get_stats())

In [5]:
# Function to get stats for particular symbol, master_stats from above. returns dictionary of all stats for coin
def get_stats(coinsymbol, master_stats): 
    time.sleep(1)
    coindict = {}
    for social_site in master_stats.keys():
        
        social_dict = {}
        for stat in master_stats[social_site]:
            try:
                social_dict[stat] = live_social_status(coinsymbol, symbol_id_dict)[social_site][stat]
            except:
                continue
        coindict[social_site] = social_dict  
    
    print('DONE WITH: ' + str(coinsymbol))
    print('------------------------')
    return coindict

# Define a worker function
def worker(queue):
    queue_full = True
    while queue_full:

        # Get data off the queue and do work
        coin = q.get()        
        data = get_stats(coin, masterstats)
        with master_lock:
            master[coin] = data
        q.task_done()

### Actually Create the Master Dictionary

In [6]:
# MASTER DICTIONARY- all data without date
master = {}

# LOCK to make sure we dont write multiple entries to the dictionary at the same time
master_lock = threading.Lock()

# Load up a queue with symbols
q = Queue()
for coin in symbols:
    q.put(coin)
    
# Start Time   
start = time.time()

# Create threads
thread_count = 10
for i in range(thread_count):
    t = threading.Thread(target=worker, args = (q,))
    t.daemon = True
    t.start()
    
# Wait until thread terminates
q.join()
print('Entire job took:',time.time() - start)

DONE WITH: ZEC
------------------------
DONE WITH: NXT
------------------------
DONE WITH: CRAIG
------------------------
DONE WITH: DOGE
------------------------
DONE WITH: BTS
------------------------
DONE WITH: LTC
------------------------
DONE WITH: YBC
------------------------
DONE WITH: PRC
------------------------
DONE WITH: XPY
------------------------
DONE WITH: XMR
------------------------
Entire job took: 5.547175884246826


### Save Data to File

In [ ]:
# Save file- if new file, enter: data, newfile=True, and filename. Will create new file and dump data into it
# If writing to old file, saves directly to MasterData.txt and adds the new data
def save_file(data, newfile=False, filename='DEFAULT.txt'):
    if newfile:
        with open(filename, 'w') as outfile:
            json.dump(data, outfile)
    else:
        try:
            json_data = json.load(open('MasterData.txt'))
            json_data[now] = data
        except IOError:
            print('FILE NOT FOUND')
            
def load_file(filename):
    json_data = json.load(open(filename))
    return json_data

now = str(datetime.datetime.now().isoformat())
save_file(master, newfile=False)